# Flask and Deep Learning Web Services

- 자동차 연비를 예측하는 model 작성  

- Flask Webserver start  

- 온라인 query를 통한 연비 예측 서비스 제공

## STEP 1. 자동차 연비를 예측하는 regression model 작성

- UCI Machine Learning Repository 의 Auto MPG dataset 을 사용

1. mpg:           continuous  
2. cylinders:     multi-valued discrete  
3. displacement:  continuous (배기량)   
4. horsepower:    continuous  
5. weight:        continuous  
6. acceleration:  continuous  
7. model year:    multi-valued discrete  
8. origin:        multi-valued discrete, 1 - USA, 2 - Europe, 3 - Japan  
9. car name:      string (unique for each instance)  

Missing Attribute Values:  horsepower has 6 missing values  ==> "?" 로 들어 있으므로 read_csv 시 nan 으로 변환

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
import pickle

In [2]:
df = pd.read_csv("data/auto-mpg.csv", na_values=['NA', '?'])
df.dropna(inplace=True)
print(df.shape)
df.head()

(392, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [4]:
df.drop('name', axis=1, inplace=True)

y = df.pop('mpg')
X = df.values

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((313, 7), (79, 7), (313,), (79,))

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
model = Sequential()
model.add(Dense(25, input_dim=X.shape[1], activation='relu'))
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', patience=5, 
                        restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_test,y_test),\
          callbacks=[monitor], verbose=2, epochs=1000)

Epoch 1/1000
10/10 - 0s - loss: 639.1387 - val_loss: 586.3868 - 293ms/epoch - 29ms/step
Epoch 2/1000
10/10 - 0s - loss: 630.0396 - val_loss: 579.0571 - 22ms/epoch - 2ms/step
Epoch 3/1000
10/10 - 0s - loss: 623.8339 - val_loss: 573.5228 - 22ms/epoch - 2ms/step
Epoch 4/1000
10/10 - 0s - loss: 619.0323 - val_loss: 569.0509 - 22ms/epoch - 2ms/step
Epoch 5/1000
10/10 - 0s - loss: 614.8302 - val_loss: 565.1352 - 23ms/epoch - 2ms/step
Epoch 6/1000
10/10 - 0s - loss: 610.8798 - val_loss: 561.4948 - 24ms/epoch - 2ms/step
Epoch 7/1000
10/10 - 0s - loss: 606.8831 - val_loss: 557.7670 - 23ms/epoch - 2ms/step
Epoch 8/1000
10/10 - 0s - loss: 602.8854 - val_loss: 553.7849 - 23ms/epoch - 2ms/step
Epoch 9/1000
10/10 - 0s - loss: 598.5718 - val_loss: 549.4495 - 22ms/epoch - 2ms/step
Epoch 10/1000
10/10 - 0s - loss: 593.7721 - val_loss: 544.5628 - 24ms/epoch - 2ms/step
Epoch 11/1000
10/10 - 0s - loss: 588.3977 - val_loss: 538.9872 - 25ms/epoch - 2ms/step
Epoch 12/1000
10/10 - 0s - loss: 582.1308 - val_lo

Epoch 97/1000
10/10 - 0s - loss: 14.1865 - val_loss: 14.2368 - 25ms/epoch - 2ms/step
Epoch 98/1000
10/10 - 0s - loss: 13.9346 - val_loss: 13.9996 - 24ms/epoch - 2ms/step
Epoch 99/1000
10/10 - 0s - loss: 13.7126 - val_loss: 13.7404 - 23ms/epoch - 2ms/step
Epoch 100/1000
10/10 - 0s - loss: 13.4947 - val_loss: 13.5134 - 23ms/epoch - 2ms/step
Epoch 101/1000
10/10 - 0s - loss: 13.3010 - val_loss: 13.2955 - 23ms/epoch - 2ms/step
Epoch 102/1000
10/10 - 0s - loss: 13.1067 - val_loss: 13.1113 - 23ms/epoch - 2ms/step
Epoch 103/1000
10/10 - 0s - loss: 12.9076 - val_loss: 12.9341 - 23ms/epoch - 2ms/step
Epoch 104/1000
10/10 - 0s - loss: 12.7380 - val_loss: 12.7293 - 23ms/epoch - 2ms/step
Epoch 105/1000
10/10 - 0s - loss: 12.5740 - val_loss: 12.5375 - 24ms/epoch - 2ms/step
Epoch 106/1000
10/10 - 0s - loss: 12.4192 - val_loss: 12.3700 - 23ms/epoch - 2ms/step
Epoch 107/1000
10/10 - 0s - loss: 12.2806 - val_loss: 12.2451 - 23ms/epoch - 2ms/step
Epoch 108/1000
10/10 - 0s - loss: 12.1335 - val_loss: 12.

Epoch 194/1000
10/10 - 0s - loss: 8.3227 - val_loss: 8.2978 - 22ms/epoch - 2ms/step
Epoch 195/1000
10/10 - 0s - loss: 8.3187 - val_loss: 8.2064 - 23ms/epoch - 2ms/step
Epoch 196/1000
10/10 - 0s - loss: 8.2788 - val_loss: 8.2100 - 21ms/epoch - 2ms/step
Epoch 197/1000
10/10 - 0s - loss: 8.2741 - val_loss: 8.2519 - 23ms/epoch - 2ms/step
Epoch 198/1000
10/10 - 0s - loss: 8.2484 - val_loss: 8.2027 - 22ms/epoch - 2ms/step
Epoch 199/1000
10/10 - 0s - loss: 8.2315 - val_loss: 8.2479 - 21ms/epoch - 2ms/step
Epoch 200/1000
10/10 - 0s - loss: 8.1980 - val_loss: 8.1679 - 23ms/epoch - 2ms/step
Epoch 201/1000
10/10 - 0s - loss: 8.1757 - val_loss: 8.1275 - 23ms/epoch - 2ms/step
Epoch 202/1000
10/10 - 0s - loss: 8.1731 - val_loss: 8.0993 - 25ms/epoch - 2ms/step
Epoch 203/1000
10/10 - 0s - loss: 8.1458 - val_loss: 8.1047 - 22ms/epoch - 2ms/step
Epoch 204/1000
10/10 - 0s - loss: 8.1374 - val_loss: 8.1006 - 22ms/epoch - 2ms/step
Epoch 205/1000
10/10 - 0s - loss: 8.1038 - val_loss: 8.0762 - 22ms/epoch - 2

- Train한 model 저장  

- scaling 정보 저장

In [17]:
model.save("./dnn/mpg_model.h5")
pickle.dump(sc, open("./dnn/standard_scaler.pkl", "wb"))

#### Web service 로 입력될 JSON 값 범위 check

Flask 웹 서비스가 입력 JSON이 유효한지 확인하기를 원합니다. 이를 위해서는 우리가 기대하는 값의 범위를 정합니다. 다음 코드는 예상 필드와 해당 범위를 출력하고 이 모든 정보를 Flask 웹 애플리케이션에 복사하는 JSON 객체로 패키징합니다. 이 코드를 사용하면 들어오는 JSON 요청을 확인할 수 있습니다.

In [18]:
cols = [x for x in df.columns if x not in ('mpg','name')]
cols

['cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'year',
 'origin']

In [20]:
#JSON 만들기
print("{")
for i,name in enumerate(cols):
    print(f'"{name}":{{"min":{df[name].min()},\
          "max":{df[name].max()}}}{"," if i<(len(cols)-1) else ""}')
print("}")

{
"cylinders":{"min":3,          "max":8},
"displacement":{"min":68.0,          "max":455.0},
"horsepower":{"min":46.0,          "max":230.0},
"weight":{"min":1613,          "max":5140},
"acceleration":{"min":8.0,          "max":24.8},
"year":{"min":70,          "max":82},
"origin":{"min":1,          "max":3}
}


- Python 코드를 작성하여 단일 자동차에 대한 모델을 호출하고 예측을 얻습니다. 

- 이 코드를 Flask 웹 애플리케이션에 그대로 복사해 사용합니다.

In [22]:
import os
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("./dnn/mpg_model.h5")
sc = pickle.load(open("./dnn/standard_scaler.pkl", "rb"))

x = np.zeros((1,7))

x[0,0] = 8 # 'cylinders', 
x[0,1] = 400 # 'displacement', 
x[0,2] = 80 # 'horsepower', 
x[0,3] = 2000 # 'weight',
x[0,4] = 19 # 'acceleration', 
x[0,5] = 72 # 'year', 
x[0,6] = 1 # 'origin'
print(x)

x_scaled = sc.transform(x)

pred = model.predict(x_scaled)
float(pred[0])

[[8.0e+00 4.0e+02 8.0e+01 2.0e+03 1.9e+01 7.2e+01 1.0e+00]]


20.91025161743164

## STEP 2. Flask Hello World

- Jupyter notebook에서 간단한 웹 서비스를 실행할 수 있습니다. Jupyter에서 Flask 웹 컨테이너를 실행합니다.  

- 플라스크는 Werkzeug를 랩핑하여 WSGI를 사용합니다. WSGI(Web Server Gateway Interface)는 웹 서버 소프트웨어와 파이썬으로 작성된 웹 응용 프로그램 간의 표준 인터페이스입니다.  

- run_simple은 개발용 built-in server 제공

In [23]:
from werkzeug.wrappers import Request, Response
from flask import Flask
import sys
from werkzeug.serving import run_simple

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 01:24:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 01:24:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 01:25:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 01:25:37] "GET / HTTP/1.1" 200 -


이 프로그램은 컴퓨터의 포트 9000에서 웹 서비스를 시작합니다. 이 셀은 계속 실행됩니다(잠긴 상태로 표시됨). 그러나 브라우저가 연결되기를 기다리고 있을 뿐입니다. 브라우저에서 다음 URL을 가리키면 Flask 웹 서비스와 상호 작용합니다.
* http://localhost:9000/ --> click 하여 Hello World! 확인

확인 후 notebook을 interupt 하여 webserver를 정지 시킵니다. 

## STEP 3. All Together - 완전한 MPG 예측 Flask application

일반적으로 JSON을 통해 웹 서비스와 상호 작용합니다. 브라우저에서 Flask 애플리케이션에 JSON 메시지를 보내고 Flask 애플리케이션은 JSON을 반환합니다. 갤런당 마일을 예측하는 신경망용 Flask 웹 애플리케이션을 생성합니다. 샘플 JSON은 다음과 같습니다.

```
{
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}
```

이 JSON 데이터를 웹 서버에 post하는 두 가지 다른 방법이 있습니다.  

- [POSTman](https://www.getpostman.com/)과 같은 유틸리티를 사용합니다.   

- Python 코드를 사용하여 JSON 메시지를 구성하고 Flask와 상호 작용합니다.  

In [27]:
from werkzeug.wrappers import Request, Response
from flask import Flask, request, jsonify
import uuid  #범용고유식별자 생성
import os
import numpy as np
from werkzeug.serving import run_simple
from tensorflow.keras.models import load_model
import pickle

app = Flask(__name__)

# 입력 data validation
EXPECTED = {
    "cylinders":{"min":3,"max":8},
    "displacement":{"min":68.0,"max":455.0},
    "horsepower":{"min":46.0,"max":230.0},
    "weight":{"min":1613,"max":5140},
    "acceleration":{"min":8.0,"max":24.8},
    "year":{"min":70,"max":82},
    "origin":{"min":1,"max":3}
}

# model and scaler load
model = load_model("./dnn/mpg_model.h5")
sc = pickle.load(open("./dnn/standard_scaler.pkl", "rb"))

@app.route("/api/mpg/", methods=['POST'])
def calc_mpg():
    content = request.json
    errors = []
    #input validity check
    for name in content:
        if name in EXPECTED:
            value = content[name]
            if value < EXPECTED[name]['min'] or value > EXPECTED[name]['max']:
                error.append(f"입력값 범위 초과 - {name}: {value}")
        else:
            error.append(f"Unexpected field - {name}")
    # missing input check
    for name in EXPECTED:
        if name not in content:
            errors.append(f"missing input - {name}")
            
    # prediction
    if len(errors) < 1:
        x = np.zeros((1, 7))
        x[0,0] = content['cylinders'] 
        x[0,1] = content['displacement']
        x[0,2] = content['horsepower']
        x[0,3] = content['weight']
        x[0,4] = content['acceleration'] 
        x[0,5] = content['year'] 
        x[0,6] = content['origin']
        
        x_scaled = sc.transform(x)
        pred = model.predict(x_scaled)
        
        mpg = float(pred[0])
        
        response = {"id": str(uuid.uuid4()), "mpg": mpg}
    else:
        response = {"id": str(uuid.uuid4()), "errors": errors}
        
    print(response)

    return jsonify(response)

if __name__ == '__main__':
    run_simple('localhost', 9000, app)
#     app.run(host='0.0.0.0', debug=True)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 01:40:22] "POST /api/mpg HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 01:40:22] "POST /api/mpg/ HTTP/1.1" 200 -


{'id': '21c79aab-4d58-45e6-8323-8c4c634b11f8', 'mpg': 22.59354019165039}


### 다른 notebook 에서 아래 code 수행

- CapstoneProject_1_python_requests Notebook 실행

```
import requests

json = {
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}

r = requests.post("http://localhost:9000/api/mpg",json=json)
if r.status_code == 200:
    print("Success: {}".format(r.text))
else: print("Failure: {}".format(r.text))
```